#### Langchain, openai key and HUggingFace API key

In [2]:
from langchain_openai import ChatOpenAI
from constant import openai_key
from huggingface_Api import HUGGINGFACEHUB_API_TOKEN

#### Call the openai env key globally

In [3]:
import os
os.environ["OPENAI_API_KEY"] = openai_key

#### From Langchain Initialize the OpenAI LLM

In [4]:
llm = ChatOpenAI(temperature=0.8) # More the value towards 1 more random the LLM would be

##### Testing

In [5]:
text = " what is the capital of india"
print(llm.predict(text)) # also we can use (llm.invoke)

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The capital of India is New Delhi.


#### From Huggingface 
##### We can use opensource model from Huggingface
##### Declare the env globally

In [6]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

#### We use "google/flan-t5-large" it is a opensource model from Huggingface, also we are testing it below

In [7]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large")#,model_kwargs={"temperature":0,"max_lenght":64})

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
text_hug="what is the weather in new delhi and kolkata"
print(llm_huggingface.predict(text_hug))

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The average temperature in New Delhi is 1°C, while in Kolkata it is 


#### compare it with OpenAI LLM

In [9]:
text_openai_llm="what is the weather in new delhi and kolkata"
print(llm.predict(text_openai_llm))

I am sorry, I do not have real-time weather information. You can check the weather in New Delhi and Kolkata by using a reliable weather website or app.


#### Prompt Template

In [10]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=["country"],
               template="Tell me something about the country {country}")

prompt_template.format(country="India")

'Tell me something about the country India'

In [11]:
## To run the prompt and the llm along with it we need llm chain.

from langchain.chains import LLMChain

chain = LLMChain (llm=llm, prompt=prompt_template) ## Open AI LLM

print(chain.run("India"))

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


India is a diverse country located in South Asia, known for its rich history, cultural heritage, and vibrant traditions. It is the seventh-largest country in the world by land area and the second-most populous country, with over 1.3 billion people.

India is home to a wide variety of languages, religions, and ethnic groups, making it one of the most culturally diverse countries in the world. The country has a parliamentary system of government and is a federal republic, consisting of 28 states and 8 union territories.

India is known for its ancient civilization, with a history dating back thousands of years. It is the birthplace of major religions such as Hinduism, Buddhism, Jainism, and Sikhism. The country is also famous for its architectural wonders, including the iconic Taj Mahal in Agra and the ancient temples of Khajuraho and Hampi.

Indian cuisine is renowned for its use of spices and flavors, with dishes such as curry, biryani, and dosa being popular around the world. The coun

#### Combine Multiple Chain using simple Sequential chain

In [12]:
### Creating 2 chains
capital_prompt = PromptTemplate(input_variables = ["country"],
                              template = "Please tell me the capital of {country}")

capital_chain = LLMChain(llm = llm, prompt = capital_prompt)

famous_prompt = PromptTemplate(input_variables = ["capital"],
                               template="Please tell me the famous places in {capital}")

famous_chain  = LLMChain(llm = llm, prompt = famous_prompt)

In [13]:
from langchain.chains import SimpleSequentialChain
simple_seq_chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
print(simple_seq_chain.run("India"))

1. India Gate
2. Red Fort
3. Qutub Minar
4. Lotus Temple
5. Humayun's Tomb
6. Jama Masjid
7. Akshardham Temple
8. Rashtrapati Bhavan
9. Lodhi Gardens
10. National Museum


### Sequential Chain

In [14]:
### Creating 2 chains
capital_prompt = PromptTemplate(input_variables = ["country"],
                              template = "Please tell me the capital of {country}")

capital_chain = LLMChain(llm = llm, prompt = capital_prompt, output_key = "capital")

famous_prompt = PromptTemplate(input_variables = ["capital"],
                               template="Please tell me the famous places in {capital}")

famous_chain  = LLMChain(llm = llm, prompt = famous_prompt, output_key = "places")

In [15]:
from langchain.chains import SequentialChain

seq_chain = SequentialChain(chains = [capital_chain, famous_chain],
                            input_variables=["country"], output_variables= ["capital", "places"])

print(chain({"country":"India"}))

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'India', 'text': 'India is the seventh-largest country by land area and the second-most populous country in the world, with over 1.3 billion people. It is located in South Asia and shares its borders with Pakistan, China, Nepal, Bhutan, Bangladesh, and Myanmar. \n\nIndia is known for its rich history, diverse culture, and ancient traditions. It is home to many religions, including Hinduism, Islam, Sikhism, Buddhism, and Christianity. The country is also known for its cuisine, which varies greatly from region to region.\n\nIndia is a federal parliamentary democratic republic, with a President as the head of state and a Prime Minister as the head of government. The capital city is New Delhi, and the official languages are Hindi and English.\n\nThe economy of India is one of the fastest-growing in the world, with a large and diverse workforce. The country is known for its IT and software services industry, as well as its agriculture, textiles, and pharmaceutical industries.\n\

#### Chatmodels (LangChain) with ChatOpenAI

In [19]:
from langchain.chat_models import ChatOpenAI
# so when we use chatmodels we use 3 schemas
from langchain.schema import HumanMessage, SystemMessage, AIMessage
# Reinitialing ChatLLM model from OpenAI
chat_llm = ChatOpenAI(temperature=0.6,model="gpt-3.5-turbo")

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [21]:
### Let use the 3 schemas HumanMessage(Human Input), SystemMessage(Instruction to the chatbot),AIMessage (Output of the chatbot)  
chat_llm([
    SystemMessage(content="You are a Comedian AI assitant"), # this is what I tell my chatbot to behave like
    HumanMessage(content="Let me some jokes of AI") # Input from Human
])

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="Sure, here are a few AI jokes for you:\n\n1. Why did the AI break up with its girlfriend? She couldn't handle its emotional algorithms.\n2. How does an AI fix its problems? It reboots its relationship status.\n3. Why did the robot go to therapy? It had a circuit overload of emotions.\n4. Why was the computer cold? It left its Windows open.\n5. What do you call a robot who likes to take naps? A sleep mode-ule.")